In [3]:
import os
import re
import numpy as np 
import pandas as pd 
import pickle, json, string, torch
from datetime import datetime

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed
from datasets import Dataset, DatasetDict, load_dataset
from transformers import BitsAndBytesConfig,HfArgumentParser,TrainingArguments, logging, DataCollatorForLanguageModeling, Trainer
from transformers import DataCollatorForTokenClassification

In [4]:
dataset = load_dataset("MalakhovIlya/NEREL")
folders = ["train", "test", "dev"]


#@title функции для изъятия сущностей
#!!! - то, что было в изначальном коде
def get_list_values(text):
    return text.split()

def replc_t_n(text):
    return re.sub("\t|\n", " ", text).strip()

def grouped_and_sort_labeled_data(annotation_file):

    # df_ann = pd.DataFrame([get_list_values(replc_t_n(i)) for i in annotation_file if ";" not in i])
    df_ann = pd.DataFrame([get_list_values(i) for i in annotation_file  if ";" not in i])
    df_ann[2] = df_ann[2].astype("int")
    df_ann[3] = df_ann[3].astype("int")


    '''
    df_ann.to_csv("df_ann.csv")
    # (1869-окончание) - если группировать, то исчезает прокуратура Бишкека как сущность
    # есть отдельно прокуратура и отдельно Бишкек
    #!!! grouped = df_ann.groupby([1, 2])[3].min().reset_index()
    grouped.to_csv("grouped.csv")
    print(grouped)
    '''
    return df_ann.sort_values(by=2)[[1,2,3]].values #!!! grouped.sort_values(by=2)[[1,2,3]].values

def split_text_on_labeled_tokens(text, labels):
    def chunk_text_labeling(text, start, end, is_ner = False):
        # print(f"start = {start}, end = {end}\n  text[start: end]={text[start: end]}")
        chunk_iter = 0
        ner_chunk = text[start: end].split()
        for part_of_chunk in ner_chunk:
            split_text.append(part_of_chunk)
            if is_ner:
                if chunk_iter == 0:
                    ner_label.append("B-"+ner)
                else:
                    ner_label.append("I-"+ner)
                chunk_iter += 1
            else:
                ner_label.append("O")
    ### inner function

    init_start = 0
    split_text = []
    ner_label = []
    for ner, start, end in labels:
        if start > init_start:

            chunk_text_labeling(text, init_start, start)
            chunk_text_labeling(text, start, end, True)
        else:
            chunk_text_labeling(text, start, end, True)
        init_start = end

    return split_text, ner_label


#@title функции для очистки токенов

''' в разделенных словах есть знаки препинания - нужно убрать
split_text_sent = ['"В', 'ходе', 'проведения', 'оперативно-профилактического', 'мероприятия', 'под', 'кодовым', 'названием', '"', 'Арсенал', '"', 'в',
'''

def starts_or_ends_with_punctuation(s):
    punct=string.punctuation
    punct+=' '
    return [(len(s)!=1 and s[0] in punct), (len(s)!=1 and s[-1] in punct)]
def clear_tokens(token, label):

  find_punct_start, find_punct_end = starts_or_ends_with_punctuation(token)
  if  find_punct_start==0 and find_punct_end==0: # token==punctuation mark
    return [token], [label]
  elif find_punct_start==1 and find_punct_end==0:
    return [token[0], token[1:]], ["O", label]
  elif find_punct_start==0 and find_punct_end==1:
    return [token[:-1], token[-1]], [label, "O"]
  else:
    if (len(token)==2): #случаи '#!'
       return [token[0], token[-1]], ["O", "O"]
    else:
      return [token[0], token[1:-1], token[-1]], ["O", label, "O"]
#пример
text = ['боеприпасов.', 'пресс-служба', '"В', '.', '"В.', "!#"]
labels=['SOMETHING', 'SOMETHING', 'O', 'O', 'O', 'O']
for i_text in text:
  print(f"{i_text}   ==>  {starts_or_ends_with_punctuation(i_text)}")
new_texts, new_labels = [], []
for i in range(len(text)):
  new_texts+=(clear_tokens(text[i], labels[i])[0])
  new_labels+=(clear_tokens(text[i], labels[i])[1])
print(new_texts)
print(new_labels)

for folder in folders:
  all_sequences = []
  all_labels = []
  df_folder = pd.DataFrame()
  for i_seq in dataset[f"{folder}"]:
    ann = i_seq['entities']
    txt = i_seq['text']

    '''проверка на пустые файлы'''
    if len(ann) == 0:
        continue

    '''разделение текста и токенов'''
    labels = grouped_and_sort_labeled_data(ann) #[['ORGANIZATION' 2501 2519], ....]
    split_text, ner_label = split_text_on_labeled_tokens(txt, labels)

    '''разделение текста на предложения'''
    start_iter = 0
    ner_label_sent = []
    split_text_sent =[]
    for i in range(len(ner_label)):
      if split_text[i].endswith('.') or split_text[i].endswith('!') or split_text[i].endswith('?'):
        ner_label_sent.append(ner_label[start_iter:i+1])
        split_text_sent.append(split_text[start_iter:i+1])
        start_iter = i+1

    '''очистка токенов
    почему: в разделенных словах есть знаки препинания - нужно убрать
    пример: ['"В', 'ходе', 'проведения',
    '''
    text_data, label_data =[], []
    for sent_label, sentence in zip(ner_label_sent, split_text_sent):
      new_texts, new_labels = [], []
      for i in range(len(sent_label)):
        new_texts+=(clear_tokens(sentence[i], sent_label[i])[0])
        new_labels+=(clear_tokens(sentence[i], sent_label[i])[1])
      text_data.append(new_texts)
      label_data.append(new_labels)

    # print(f"final text = {text_data}")
    # print(f"final label= {label_data}")

    df_folder = pd.concat([df_folder,pd.DataFrame({"tokens": text_data, "labels": label_data})])

  df_folder = df_folder.reset_index().drop('index', axis=1)

  # '''файлы для дообучения моделей deeppavlov'''
  # file_tr = open(f'/content/deeppavlov_tuning/{folder}.txt', "w")
  # for row in range(len(df_folder)):
  #   for i in range(len(df_folder['sequences'][row])):
  #     file_tr.writelines(f"{df_folder['sequences'][row][i]} {df_folder['labels'][row][i]}\n")
  #   file_tr.writelines(f"\n")
  # file_tr.close()

  with open(f'C:/Users/User/Desktop/PhD/changed_NEREL/{folder}_data.pickle', 'wb') as f:
    pickle.dump(df_folder, f)
    df_folder.to_csv(f"C:/Users/User/Desktop/PhD/changed_NEREL/df_{folder}.csv")
  print(f"For folder <{folder}> prepared <{df_folder.shape[0]}> sequences")

боеприпасов.   ==>  [False, True]
пресс-служба   ==>  [False, False]
"В   ==>  [True, False]
.   ==>  [False, False]
"В.   ==>  [True, True]
!#   ==>  [True, True]
['боеприпасов', '.', 'пресс-служба', '"', 'В', '.', '"', 'В', '.', '!', '#']
['SOMETHING', 'O', 'SOMETHING', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
For folder <train> prepared <8979> sequences
For folder <test> prepared <1193> sequences
For folder <dev> prepared <1138> sequences


In [5]:
file = open('C:/Users/User/Desktop/PhD/changed_NEREL/train_data.pickle', 'rb')
train_data = pickle.load(file)
file.close()

file1 = open('C:/Users/User/Desktop/PhD/changed_NEREL/test_data.pickle', 'rb')
test_data = pickle.load(file1)
file1.close()

file2 = open('C:/Users/User/Desktop/PhD/changed_NEREL/dev_data.pickle', 'rb')
dev_data = pickle.load(file2)
file2.close()

In [78]:
entity="FAMILY"

# зануление ненужных сущностей
def replace_tags(tags, entity):
    return ['O' if tag not in ['O', f'B-{entity}', f'I-{entity}'] else tag for tag in tags]


def get_needed_entity(entity):
    few_shot_text = dev_data[dev_data['labels'].apply(lambda x: f'B-{entity}' in x)]
    few_shot_text['labels'] = few_shot_text.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
    
    few_shot_text=few_shot_text.reset_index().drop("index", axis=1)
   

    few_shot_text1 = test_data[test_data['labels'].apply(lambda x: f'B-{entity}' in x)]
    few_shot_text1['labels'] = few_shot_text1.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
    few_shot_text1=few_shot_text1.reset_index().drop("index", axis=1)
    

    few_shot_check = train_data[train_data['labels'].apply(lambda x: f'B-{entity}' in x)]
    few_shot_check['labels'] = few_shot_check.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
    few_shot_check=few_shot_check.reset_index().drop("index", axis=1)
    return  few_shot_text, few_shot_text1, few_shot_check
few_shot_text, few_shot_text1, few_shot_check = get_needed_entity(entity)


def extract_entity(df, entity):
    entities_list=[]
    for i in range(len(df)):
        entities = []
        bool_check=False
        for i_word in range(len(df.iloc[i]['tokens'])):
            if df.iloc[i]['labels'][i_word]==f'B-{entity}':
                bool_check=True
                find_entity = df.iloc[i]['tokens'][i_word]
            elif df.iloc[i]['labels'][i_word]==f'I-{entity}':
                find_entity += " "  + df.iloc[i]['tokens'][i_word]
            elif df.iloc[i]['labels'][i_word]=='O' and bool_check==True:
                bool_check=False
                entities.append(find_entity)       
        entities_list.append(entities)         
    df["entities"] = entities_list
    df['joined_entities'] = df['entities'].apply(lambda x: ', '.join(x))
    return df

few_shot_check = extract_entity(few_shot_check, entity)
few_shot_text = extract_entity(few_shot_text, entity)
few_shot_text1 = extract_entity(few_shot_text1, entity)


def remove_repeated_subsequences(text):
    words = text.split()
    n = len(words)
    i = 0
    result = []
    
    while i < n:
        found_repetition = False
        
        for length in range(1, (n - i) // 2 + 1):
            subsequence = words[i:i + length]
            next_subsequence = words[i + length:i + 2 * length]
            if subsequence == next_subsequence:
                found_repetition = True
                i += length
                break
        
        if not found_repetition:
            result.append(words[i])
            i += 1

    return ' '.join(result)


res2 = pd.concat([few_shot_text1, few_shot_text], ignore_index=True) 
res1 = pd.concat([res2, few_shot_check], ignore_index=True) 
print(f"кол-во предложений с сущностью {entity} = {len(few_shot_check)+len(few_shot_text1)+len(few_shot_text)}")
df_example = pd.DataFrame()
labels = [5, 14, 20, 26, 37]
for i in labels:
    df_example = df_example._append(res1.iloc[i], ignore_index=True)
df_example
df_check = res1.drop(labels=labels, axis=0).reset_index().drop("index", axis=1)
df_check.to_csv("df_fewshot.csv")
df_check

кол-во предложений с сущностью FAMILY = 39


C:\Users\User\AppData\Local\Temp\ipykernel_25700\363986054.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  few_shot_text['labels'] = few_shot_text.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_25700\363986054.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  few_shot_text1['labels'] = few_shot_text1.apply(lambda x: replace_tags(x['labels'], entity),axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_25700\363986054.py:21: SettingWithCopyWarning: 

,tokens,labels,entities,joined_entities
0,"[Родившийся, 12, июня, 1915, года, американски...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[Рокфеллеров],Рокфеллеров
1,"[По, версии, следователей, ,, Скрипалей, в, Ве...","[O, O, O, O, B-FAMILY, O, O, O, O, O, O, O, O,...",[Скрипалей],Скрипалей
2,"[В, отравлении, отравлении, Скрипалей, Скрипал...","[O, O, O, O, B-FAMILY, O, O, O, O, O, O, O, O,...",[Скрипалей],Скрипалей
3,"[Ранее, Bellingcat, и, The, Insider, назвали, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-FAMI...",[Скрипалей],Скрипалей
4,"[Журналисты, не, смогли, установить, точное, з...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[Скрипалей],Скрипалей
5,"[Напомним, ,, как, ранее, сообщало, ИА, REGNUM...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[Харрисонов],Харрисонов
6,"[Благотворительный, фонд, Гейтсев, Гейтсев, по...","[O, O, O, B-FAMILY, O, O, O, O, O, O]",[Гейтсев],Гейтсев
7,"[Семья, Обамы, Обамы, приобрела, дом, в, Вашин...","[B-FAMILY, I-FAMILY, O, O, O, O, O, O, O, O, O...",[Семья Обамы],Семья Обамы
8,"[После, окончания, срока, работы, на, посту, п...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[Тюдоров],Тюдоров
9,"[9, апреля, семья, капитана, Мура, капитана, М...","[O, O, B-FAMILY, I-FAMILY, I-FAMILY, O, O, O, ...",[семья капитана Мура],семья капитана Мура


In [ ]:
# OLD
needed_row_df=[]
for i in range(len(needed_rows)):
    needed_word=[]
    for i_word in range(len(needed_rows.iloc[i]['labels'])):
        if re.match("[B|I]-FAMILY", needed_rows.iloc[i]['labels'][i_word])!=None:
            needed_word.append(needed_rows.iloc[i]['tokens'][i_word])
    needed_row_df.append(needed_word)
needed_rows['words'] = needed_row_df 
needed_rows

In [ ]:
# OLD
entities_list=[]
for i in range(len(needed_rows)):
    entities = []
    for i_word in range(len(needed_rows.iloc[i]['tokens'])):
        if needed_rows.iloc[i]['labels'][i_word]!='O':
            if needed_rows.iloc[i]['labels'][i_word]=='B-FAMILY' or  needed_rows.iloc[i]['labels'][i_word]=='I-FAMILY':
                entities.append(needed_rows.iloc[i]['tokens'][i_word])
    entities_list.append(' '.join(entities))
            
needed_rows.loc[:, "entities"] = entities_list
needed_rows

In [77]:
pd.read_csv("C:/Users/User/Desktop/PhD/prompt_hypothesis - prompt1.csv", encoding='utf-8')['generation_1'][3].split('\n')

['### Твоя задача найти слова или словосочетания (сущности), которые относятся к термину СЕМЬЯ. СЕМЬЯ — это малая социальная группа, основанная на браке, кровном родстве или усыновлении (удочерении). ',
 '',
 '    ### Пример нахождения сущности СЕМЬЯ в тексте',
 '    текст: Оппозиционный саудовский журналист Джамал Хашкаджи , известный критикой в адрес королевской семьи Саудовской Аравии , пропал 2 октября после того , как пришёл в генконсульство в Стамбуле за документами . ',
 '    сущности СЕМЬЯ: королевской семьи Саудовской Аравии',
 '',
 '    текст: Дочь Барака Обамы будет учиться в Гарвардском университете Семья Обама : Малия , Мишель , Барак , Саша ( Наташа ) Старшая дочь президента Барака Обамы Малия сделает годичный перерыв после окончания школы , после чего в 2017 году начнёт обучение в престижном Гарвардском университете . ',
 '    сущности СЕМЬЯ: Семья Обама',
 '',
 '    текст: Праправнук Николая I по мужской младшей линии ; принадлежал к ветви « Николаевичей » рода Романовы

In [82]:
def get_entities(df):
    res=[]
    for i_output in range(len(df)):
        res.append(df['generation_1'][i_output].split('\n')[22])
    df['generate_output'] = res
    return df
data_output_h1 = pd.read_csv("C:/Users/User/Desktop/PhD/prompt_hypothesis - prompt2.csv", encoding='utf-8')

data_output_h1 = get_entities(data_output_h1)
data_output_h1['real_output'] = df_check['joined_entities']
data_output_h1

,#,model,prompt,generation_1,generation_2,generations_other,max_new_tokens,num_beams,temperature,top_p,top_k,exp_descr,generate_output,real_output
0,34,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #0,к: Рокфеллер,Рокфеллеров
1,35,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #1,: Скрипалей,Скрипалей
2,36,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #2,т:,Скрипалей
3,37,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #3,:,Скрипалей
4,38,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #4,"же,",Скрипалей
5,39,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #5,"лишь, семья, Харрисонов",Харрисонов
6,40,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #6,т: Гейтсев,Гейтсев
7,41,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #7,лишь,Семья Обамы
8,42,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #8,ли,Тюдоров
9,43,/kaggle/input/mistral/pytorch/7b-v0.1-hf/1,### Ты извлекаешь ключевые слова связанные со ...,### Ты извлекаешь ключевые слова связанные со ...,NaN,NaN,100,1,1,0,0,family prompt2 #9,: семья капитана Мура,семья капитана Мура


In [84]:
data_output_h1.to_csv("prompt2_answer_0_502.csv")

In [83]:
def calculate_f1_score(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    # print(f"set1 = {set1}, set2={set2}")
    TP = len(set1.intersection(set2))
    FP = len(set2 - set1)
    FN = len(set1 - set2)
    # print(f"TP={TP}, FP={FP}, FN={FN}\nset1 - set2={set1.intersection(set2)}")
    
    
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    # print(f"precision = {precision}, recall = {recall}")
   
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0    
    return f1_score

f1_family=[]
for i in range(len(data_output_h1)):
    f1_family.append(calculate_f1_score(data_output_h1['generate_output'][i], data_output_h1['real_output'][i]))
print(np.average(f1_family))

0.5021209613054027


In [17]:
import numpy as np
# from sklearn.metrics import f1_score
from seqeval.metrics import f1_score, classification_report
y_true = few_shot_check.iloc[6]['entities']
y_pred = few_shot_check.iloc[6]['zero_greedy']
f1_score(y_true, y_pred, average='macro')

TypeError: Found input variables without list of list.

In [6]:
def calculate_f1_score(list1, list2):
    # Convert lists to sets for faster comparison
    set1 = set(list1)
    set2 = set(list2)
    # print(f"set1 = {set1}, set2={set2}")
    
    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
    TP = len(set1.intersection(set2))
    FP = len(set2 - set1)
    FN = len(set1 - set2)
    # print(f"TP={TP}, FP={FP}, FN={FN}\nset1 - set2={set1.intersection(set2)}")
    
    # Calculate Precision and Recall
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    # print(f"precision = {precision}, recall = {recall}")
    # Calculate F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    
    return f1_score
# y_true = few_shot_check.iloc[6]['entities']
# y_pred = few_shot_check.iloc[6]['zero_greedy']
# calculate_f1_score(y_true, y_pred)
# # Example usage:
# f1_beam0, f1_greedy0, f1_sample0 = [], [], []
# f1_beam_few, f1_greedy_few, f1_sample_few = [], [], []
# for i in range(len(few_shot_check)):
#     list_word = few_shot_check.iloc[i]['entities']
#     list_greedy0 = few_shot_check.iloc[i]['zero_greedy']
#     list_beam0 = few_shot_check.iloc[i]['zero_beam']
#     list_sample0 = few_shot_check.iloc[i]['zero_sample']

#     f1_beam0.append(calculate_f1_score(list_word, list_beam0))
#     f1_greedy0.append(calculate_f1_score(list_word, list_greedy0))
#     f1_sample0.append(calculate_f1_score(list_word, list_sample0))


#     list_greedy_few = few_shot_check.iloc[i]['few_greedy']
#     list_beam_few = few_shot_check.iloc[i]['few_beam']
#     list_sample_few = few_shot_check.iloc[i]['few_sample']

#     f1_beam_few.append(calculate_f1_score(list_word, list_beam_few))
#     f1_greedy_few.append(calculate_f1_score(list_word, list_greedy_few))
#     f1_sample_few.append(calculate_f1_score(list_word, list_sample_few))
# print(f"zero shot: beam, greedy, sample")
# print(np.average(f1_beam0), np.average(f1_greedy0), np.average(f1_sample0))
# print(f1_beam0, f1_greedy0, f1_sample0)

# print(f"few shot: beam, greedy, sample")
# print(np.average(f1_beam_few), np.average(f1_greedy_few), np.average(f1_sample_few))
# print(f1_beam_few, f1_greedy_few, f1_sample_few)

In [18]:
df_llm = pd.read_csv(f"C:/Users/User/Downloads/check_person_llm - Лист1.csv")
df_bert = pd.read_csv(f"C:/Users/User/Downloads/data_1.csv")
df_llm
f1_bert, f1_llm =[], []
for i in range(len(df_llm)):
    f1_bert.append(calculate_f1_score(df_bert['entity'][i], df_bert['person'][i]))
    f1_llm.append(calculate_f1_score(df_llm['entity'][i], df_llm['llm'][i]))
print(np.average(f1_bert), np.average(f1_llm))

0.7474293723039138 0.9094823454506431


In [19]:
check_few_shot = pd.read_csv(f"C:/Users/User/Downloads/df_fewshot_rrr - df_fewshot.csv.csv")
check_few_shot

,Unnamed: 0,tokens,labels,entities,greedy,beam_5,beam_25,sampling
0,0,"['Родившийся', '12', 'июня', '1915', 'года', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['Рокфеллеров'],"['тикер' , 'государственный деятель' , 'глобал...","['тикер', 'стандарт ойил', 'висепрезентиенс', ...",[''],"['тский банкир', 'государственный деятель' , '..."
1,1,"['По', 'версии', 'следователей', ',', 'Скрипал...","['O', 'O', 'O', 'O', 'B-FAMILY', 'O', 'O', 'O'...",['Скрипалей'],[''],[''],[''],['']
2,2,"['В', 'отравлении', 'Скрипалей', 'отравлении',...","['O', 'O', 'O', 'O', 'B-FAMILY', 'O', 'O', 'O'...",['Скрипалей'],[''],[''],[''],['']
3,3,"['Ранее', 'Bellingcat', 'и', 'The', 'Insider',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['Скрипалей'],[''],[''],[''],['']
4,4,"['Журналисты', 'не', 'смогли', 'установить', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['Скрипалей'],[''],[''],[''],['']
5,5,"['Напомним', ',', 'как', 'ранее', 'сообщало', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['Харрисонов'],[''],[''],[''],['']
6,6,"['Благотворительный', 'фонд', 'Гейтсев', 'Гейт...","['O', 'O', 'O', 'B-FAMILY', 'O', 'O', 'O', 'O'...",['Гейтсев'],[''],[''],[''],['']
7,7,"['Семья', 'Обамы', 'Обамы', 'приобрела', 'дом'...","['B-FAMILY', 'I-FAMILY', 'O', 'O', 'O', 'O', '...",['Семья Обамы'],"['Семья Обамы', 'Барак Обама', 'Мишель']",[''],[''],['']
8,8,"['После', 'окончания', 'срока', 'работы', 'на'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['Тюдоров'],[''],[''],[''],['']
9,9,"['9', 'апреля', 'семья', 'капитана', 'Мура', '...","['O', 'O', 'B-FAMILY', 'I-FAMILY', 'I-FAMILY',...",['семья капитана Мура'],[''],[''],[''],['']


In [23]:
check_few_shot = pd.read_csv(f"C:/Users/User/Downloads/df_fewshot_rrr - df_fewshot.csv.csv")
a, b, c, d, e =[], [], [], [], []
for i in range(len(check_few_shot)):
    a.append(calculate_f1_score(check_few_shot['entities'][i], check_few_shot['greedy'][i]))
    b.append(calculate_f1_score(check_few_shot['entities'][i], check_few_shot['beam_5'][i]))
    c.append(calculate_f1_score(check_few_shot['entities'][i], check_few_shot['beam_25'][i]))
    d.append(calculate_f1_score(check_few_shot['entities'][i], check_few_shot['sampling'][i]))

print(f'значение f1 few-shot (greedy) = {np.average(a)}, значение f1 few-shot (beam_5) = {np.average(b)}, значение f1 few-shot (beam_25) = {np.average(c)}, значение f1 few-shot (sampling) = {np.average(d)},') 
print(np.average(a), np.average(b), np.average(c), np.average(d))

значение f1 few-shot (greedy) = 0.4939736486377785, значение f1 few-shot (beam_5) = 0.38258791954977, значение f1 few-shot (beam_25) = 0.37227811334821087, значение f1 few-shot (sampling) = 0.4144720526109307,
0.4939736486377785 0.38258791954977 0.37227811334821087 0.4144720526109307


In [9]:
df_llm

,text,entity,llm
0,"Надеюсь, что люди заглянут дальше названия и в...",[Глин],[Глин]
1,Среди погибших были женщины и дети сказал Шаус...,"[Шауста Ян Ахади, Shausta Jan Ahady]",[Шауста Ян Ахади]
2,Да Силва ещё может обжаловать вердикт суда.,[Да Силва],[Да Силва]
3,Мария Николаевна Антонова родилась 24 августа ...,[Мария Николаевна Антонова],[Антонова Мария Николаевна]
4,"Антошка, Антошка, пойдем копать картошку, тили...","[Антошка, Антошка]",[Антошка]
5,"Женёк пригласил нас на пикник в субботу, обеща...",[Женёк],[женёк]
6,"Сам Эней меж­ду тем, на утес взо­брав­шись выс...",[Эней],[Эней]
7,До него этим титулом обладали американец Эванд...,"[Эвандер Холифилд, О’Нил Белл]","[Эвандер Холифилд, О’Нил Белл]"
8,"В 1937 году Самбук представляет работу ""Растит...",[Самбук],[Самбук]
9,Что касается Березовского и рекламы.,[Березовского],[Березовский]


In [20]:
few_shot_check.to_csv("1.csv")

In [ ]:
list_text = open(r"C:\Users\User\Desktop\PhD\output_beam\kaggle\working\output_beam\9.txt", encoding='utf-8').readlines()
print(re.findall('[а-яА-Я]+', ' '.join(list_text[3:]).split('\n \n')[0]))